# Probably the simplest BIP in the world (the short story)

Here we define and solve a very simple Bayesian inverse problem (BIP) using 
CUQIpy. The purpose of this notebook is to introduce the basic concepts of
Bayesian inverse problems in a simple way and to use minimal CUQIpy code to
solve it.

In the next notebooks (the long story), we discuss more details about setting up
this BIP in CUQIpy and provide many exercises to help the reader to explore 
using CUQIpy in solving BIPs and think of slightly different BIP modeling 
scenarios.


## <font color=#CD853F> Contents of this notebook: </font>
  * [0. Learning objectives](#r-learning-objectives)
  * [1. Defining the BIP](#r-defining-the-bip)
      * [1.1. The forward model](#r-the-forward-model)
      * [1.2. The prior](#r-the-prior)
      * [1.3. The noise distribution](#r-the-noise-distribution)
      * [1.4. The data distribution](#r-the-data-distribution)
      * [1.5. The likelihood function](#r-the-likelihood-function)
      * [1.6. Putting it all together, the BIP](#r-putting-it-all-together-the-bip)

  * [2. Solving the BIP](#r-solving-the-bip)
      * [2.2. Maximum a posteriori (MAP) estimate](#r-maximum-a-posteriori-map-estimate)
      * [2.3. Sampling from the posterior](#r-sampling-from-the-posterior)

  * [3. Summary](#r-summary)

  * [References](#r-references)



## <font color=#CD853F> 0. Learning objectives </font> <a name="r-learning-objectives"></a>
  * Create a simple BIP in CUQIpy
      * Create a linear forward model object
      * Create distribution objects that represent the prior, the noise, and the data distributions
      * Use the `BayesianProblem` class to define the BIP

  * Solve a simple BIP in CUQIpy
      * Compute the maximum a posteriori (MAP) estimate
      * Sample from a simple posterior distribution and visualize the results

## <font color=#CD853F> 1. Defining the BIP </font> <a name="r-defining-the-bip"></a>


Consider the following inverse problem: given observed data $b$, determine $x_1$, and $x_2$:

$$b = x_1 + x_2 + e \;\;\mathrm{with}\;\; e \sim \mathrm{Gaussain}(0, 0.1)$$

We can write it as:

$$b = \mathbf{A}\mathbf{x} + e = \large(1,1\large)\binom{x_1}{x_2} + e$$


|variable     |    description          |dimension      |
|:------------|:------------------------|:--------------|
|$\mathbf{x}$ |parameter to be inferred |2-dimensional​  |
|$\mathbf{A}$ |forward model            |1-by-2 matrix  |
|$b$          |data                     | 1-dimensional |
|$e$          |noise                    |1-dimensional  ​|

This problem is:
* Considered a Linear inverse problem since the forward model is linear.
* Ill-posed (in the sense of Hadamard [1]) since the solution is not unique, i.e., for some given value of $b$, e.g., $b=3$, all points $(x_1, x_2)$ that satisfy $x_1 + x_1 = 3$ are solutions to the (noise-free) problem.

Let us define the BIP components and solve it using CUQIpy.

In [ ]:
# Importing the required libraries
from cuqi.distribution import Gaussian
from cuqi.problem import BayesianProblem
from cuqi.model import LinearModel
import numpy as np
import matplotlib.pyplot as plt

### <font color=#8B4513> 1.1. The forward model </font> <a name="r-the-forward-model"></a>

Let us define the forward model $\mathbf{A}$, we first define the matrix:

In [ ]:
A_matrix = np.array([[1.0, 1.0]])

Now we wrap `A_matrix` in a `CUQIpy` forward model object, we name `A`, as follows:

In [ ]:
A = LinearModel(A_matrix)
print(A)

### <font color=#8B4513> 1.2. The prior </font> <a class="anchor" id="r-the-prior"></a>

**Bayesian approach: Use prior to express belief about solution**

A common choice for simplicity is the zero mean Gaussian prior where the components are independent and identically distributed (i.i.d.):

$$ \mathbf{x} \sim \mathrm{Gaussian}(\mathbf{0}, \delta^2 \mathbf{I})$$

The probability density function (PDF) of such a Gaussian prior is expressed as

$$ \pi (\mathbf{x}) = \frac{1}{\sqrt{(2 \pi)^n \delta^{2n}}} \mathrm{exp}\left(-\frac{||\mathbf{x}||^2}{2\delta^2}\right) $$

where:
- $n$ is the dimension of the parameter space (which is 2 in this specific case),
- $\delta$ is the standard deviation of the prior distribution,
- $\mathbf{I}$ is the identity matrix.

Let us define the prior distribution for the parameters $x_1$ and $x_2$:

In [ ]:
x = Gaussian(np.zeros(2), 2.5)
print(x)

In [ ]:
def plot2d(val, x1_min, x1_max, x2_min, x2_max, N2=201):
    # plot
    pixelwidth_x = (x1_max-x1_min)/(N2-1)
    pixelwidth_y = (x2_max-x2_min)/(N2-1)

    hp_x = 0.5*pixelwidth_x
    hp_y = 0.5*pixelwidth_y

    extent = (x1_min-hp_x, x1_max+hp_x, x2_min-hp_y, x2_max+hp_y)

    plt.imshow(val, origin='lower', extent=extent)
    plt.colorbar()


def plot_pdf_2D(distb, x1_min, x1_max, x2_min, x2_max, N2=201):
    N2 = 201
    ls1 = np.linspace(x1_min, x1_max, N2)
    ls2 = np.linspace(x2_min, x2_max, N2)
    grid1, grid2 = np.meshgrid(ls1, ls2)
    distb_pdf = np.zeros((N2,N2))
    for ii in range(N2):
        for jj in range(N2):
            distb_pdf[ii,jj] = np.exp(distb.logd([grid1[ii,jj], grid2[ii,jj]])) 
    plot2d(distb_pdf, x1_min, x1_max, x2_min, x2_max, N2)



We can plot the prior PDF for the parameters $x_1$ and $x_2$, along with samples from the prior distribution:

In [ ]:
# Plot PDF
plot_pdf_2D(x, x1_min=-5, x1_max=5, x2_min=-5, x2_max=5)

# Sample
x_samples = x.sample(1000)

# Plot samples
x_samples.plot_pair(ax=plt.gca())
plt.ylim(-5,5);
plt.xlim(-5,5);


### <font color=#8B4513> 1.3. The noise distribution </font> <a class="anchor" id="r-the-noise-distribution"></a>

- As mentioned earlier, we assume $e \sim \mathrm{Gaussain}(0, 0.1)$ 
- We can define the noise distribution as follows:

In [ ]:
e = Gaussian(0, 0.1)

Let us also plot the PDF of the noise distribution, using the python function `plot_pdf_1D`:

In [ ]:

def plot_pdf_1D(distb, min, max):
    grid = np.linspace(min, max, 1000)
    y = [distb.pdf(grid_point) for grid_point in grid]
    plt.plot(grid, y)

In [ ]:
plot_pdf_1D(e, -1.5, 1.5)

### <font color=#8B4513> 1.4. The data distribution </font> <a class="anchor" id="r-the-data-distribution"></a>

The noise in the measurement data follows $e \sim \mathrm{Gaussain}(0, 0.1)$ and due to the relation $b = \mathbf{A}\mathbf{x} + e $, we can write

$$ b | \mathbf{x} \sim \mathrm{Gaussian}(\mathbf{A}\mathbf{x}, \sigma^2\mathbf{I})$$

and in this case we specify $\sigma^2 = 0.1$.

$$ \pi (b | \mathbf{x}) = \frac{1}{\sqrt{(2 \pi)^m \sigma^{2m}}} \mathrm{exp}\left(-\frac{||\mathbf{A}\mathbf{x}- b||^2}{2\sigma^2}\right) $$

- The data distribution is the conditional distribution of $b$ given $\mathbf{x}$.
- This PDF can only be evaluated for a given $\mathbf{x}$.

We create the data distribution object as follows:

In [ ]:
b = Gaussian(A@x, 0.1)
print(b)

Before sampling or evaluating the PDF of `b`, we need to specify the value of the parameters `x`, let us choose the following value:

In [ ]:
particular_x = np.array([1.5, 1.5])

Then we condition the data distribution `b` on the given parameter `particular_x`:

In [ ]:
b_given_particular_x = b(x=particular_x)
print(b_given_particular_x)

Now we have the distribution object `b_given_particular_x` that represents the data distribution given a particular value of the parameters `x`. We can now plot the PDF of this distribution:

In [ ]:
plot_pdf_1D(b_given_particular_x, 1.5, 4.5)

We can use `b_given_particular_x` to simulate noisy data assuming that the true `x` parameters is `particular_x`:

In [ ]:
b_obs = b_given_particular_x.sample()
print(b_obs)

### <font color=#8B4513> 1.5. The likelihood function </font> <a class="anchor" id="r-the-likelihood-function"></a>


We obtain the likelihood function by fixing observed data $b^\mathrm{obs}$ in the data distribution and considering the function of $\mathbf{x}$:

$$L (\mathbf{x} | b^\mathrm{obs}) \mathrel{\vcenter{:}}= \pi (b^\mathrm{obs} | \mathbf{x})$$


Example, given observed data  $b^\mathrm{obs} $

$$L (x_1, x_2 | b=b^\mathrm{obs}) = \frac{1}{\sqrt{2 \pi \cdot 0.1}} \mathrm{exp}\left(-\frac{(x_1+x_2- b^\mathrm{obs})^2}{2\cdot 0.1}\right) $$




In CUQIpy, we can define the likelihood function as follows:

In [ ]:
likelihood = b(b=b_obs)
print(likelihood)

Note that the likelihood function is a density function and is not a distribution. If we try to compute `pdf` for example, we will get an error:

We plot the likelihood function for the observed data `b_obs`:

In [ ]:
x1_lim = np.array([-5, 5])
x2_lim = np.array([-5, 5])
plot_pdf_2D(
    likelihood,
    x1_min=x1_lim[0], x1_max=x1_lim[1],
    x2_min=x2_lim[0], x2_max=x2_lim[1])

**Note:** One method of estimating the inverse proble solution is to maximize the likelihood function, which is equivalent to minimizing the negative log-likelihood function. This is known as the maximum likelihood (ML) point estimate.

For this problem, all the points $(x_1, x_2)$ that satisfy $x_1 + x_2 = b^\mathrm{obs}$ are solutions to this maximization problem. These solutions are depicted as the red line in the likelihood plot below.

In [ ]:
# Plot the likelihood
plot_pdf_2D(
    likelihood,
    x1_min=x1_lim[0], x1_max=x1_lim[1],
    x2_min=x2_lim[0], x2_max=x2_lim[1])

# Plot the line x2 = b_obs - x1
plt.plot(x1_lim, b_obs-x1_lim, '--r')
plt.ylim(x2_lim);

Combining the likelihood with the prior will give us a unique maximum a posteriori (MAP) point estimate as we will see next.

### <font color=#8B4513> 1.6. Putting it all together, the BIP </font> <a class="anchor" id="r-putting-it-all-together-the-bip"></a>

#### Posterior definition using the Bayes’ rule
- The posterior is proportional to the product of **likelihood** and **prior**

$$\pi(\mathbf{x} | b) \propto \pi( b|\mathbf{x})\pi(\mathbf{x})$$

- Note that $\pi( b|\mathbf{x})$ here denotes the likelihood and not the data distribution, despite often written that way.



In CUQIpy, we can use the class `BayesianProblem` to bundle the prior, the data distribution, and the data, then use it to explore the posterior distribution (e.g. point estimate and sampling):

In [ ]:
BP = BayesianProblem(b, x)
print(BP)

Now we pass the data:

In [ ]:
BP.set_data(b=b_obs)
print(BP)

Note the difference in the target of the `BayesianProblem` object before and after passing the data.

## <font color=#CD853F> 2. Solving the BIP </font> <a class="anchor" id="r-solving-the-bip"></a>

### <font color=#8B4513> 2.1. Maximum a posteriori (MAP) estimate </font> <a class="anchor" id="r-maximum-a-posteriori-map-estimate"></a>
 
- Maximizer of the posterior

$$\mathbf{x}^* = \underset{\mathbf{x}}{\operatorname{argmax\;}} \pi(\mathbf{x} | b)$$
- In the case with Gaussian noise and Gaussian prior, this is the classic Tikhonov solution

$$\mathbf{x}^* = \underset{\mathbf{x}}{\operatorname{argmin\;}} \frac{1}{2 \sigma^2} ||\mathbf{A}\mathbf{x}- b^\mathrm{obs}||_2^2 + \frac{1}{2\delta^2}||\mathbf{x} ||^2_2$$




To compute (and print) the MAP estimate in CUQIpy, we write:

In [ ]:
map_estimate = BP.MAP()
print(map_estimate)

### <font color=#8B4513> 2.2. Sampling from the posterior </font> <a class="anchor" id="r-sampling-from-the-posterior"></a> 

The MAP point is a very useful point estimate, but it does not provide information about the uncertainty in the estimate. Thus, we need to sample from the posterior distribution to quantify the uncertainty.

To sample the posterior distribution in CUQIpy, we can use the `sample_posterior` method of the `BayesianProblem` object:

In [ ]:
samples = BP.sample_posterior(1000)

We visualize the samples:

In [ ]:
# Plot the posterior PDF
plot_pdf_2D(BP.posterior, x1_lim[0], x1_lim[1], x2_lim[0], x2_lim[1])

# Plot the posterior samples
samples.plot_pair(ax=plt.gca())
plt.plot(map_estimate[0], map_estimate[1], 'ro')
plt.ylim(x2_lim);
plt.xlim(x1_lim);

## <font color=#CD853F> 3. Summary </font> <a class="anchor" id="r-summary"></a>
In this notebook, we defined a simple Bayesian inverse problem (BIP) of two unknown parameters and solved it using CUQIpy. We defined the forward model, the prior, and the data distribution; and created simulated data. We then combined these components to define the BIP using the `BayesianProblem` class. We computed the maximum a posteriori (MAP) estimate and sampled from the posterior distribution to quantify the uncertainty in the estimate.

## <font color=#CD853F> References </font> <a class="anchor" id="r-references"></a>
1. Latz, J. (2020). On the well-posedness of Bayesian inverse problems. SIAM/ASA Journal on Uncertainty Quantification, 8(1), 451-482.